# Maryland Car Accident Analysis

### Note
Data file 'maryland_accident_dataset_by_report_no.csv' has already been trimmed.
1. A single individual record is registered for each report case.
2. Records including NaN are deleted.
3. All individual registered are drivers. (PERSON_TYPE == 'D')

## Purpose
Differentiate car accident cases.  
Understand causes of casuality.  
Design solutions.

## Process
Preprocessing  
Clustering  
Classifying    
Insight

### Preprocessing

In [1]:
# Import packages
import pandas as pd
import collections

In [2]:
# Read data as DataFrame
df = pd.read_csv('maryland_accident_dataset_by_report_no.csv')
# df = pd.read_csv('maryland_1year_nan_contained.csv', low_memory=False)
# Check total records
len(df.index)

29593

In [3]:
# To make all columns shown, find out the number of columns
n = len(df.columns)
# Change display setting
pd.options.display.max_columns = n

In [4]:
# Copy a new dataframe to modify
cdf = df.copy()

#### Drop attributes
Remaining attributes should satisfy following conditions.
1. It tells the driver's state.
2. It tells the environmental condition.
3. It tells the motion of the car at and right before the accident.
4. It is not skewed.

In [5]:
# Get all column names as a list
cdf.columns

Index(['REPORT_NO', 'CDL_FLAG', 'CONDITION_CODE', 'EQUIP_PROB_CODE',
       'FAULT_FLAG', 'INJ_SEVER_CODE', 'PERSON_ID', 'PERSON_TYPE',
       'SAF_EQUIP_CODE', 'SEX_CODE', 'VEHICLE_ID', 'COLLISION_TYPE_CODE',
       'C_M_ZONE_FLAG', 'JUNCTION_CODE', 'LANE_CODE', 'LIGHT_CODE',
       'RD_COND_CODE', 'RD_DIV_CODE', 'SURF_COND_CODE', 'WEATHER_CODE',
       'AREA_DAMAGED_CODE_MAIN', 'BODY_TYPE_CODE', 'DAMAGE_CODE',
       'HIT_AND_RUN_FLAG', 'MOVEMENT_CODE', 'TIME', 'AGE'],
      dtype='object')

In [6]:
# Check if PERSON_TYPE is filled with a single value
collections.Counter(cdf.PERSON_TYPE)

Counter({'D': 29593})

In [7]:
# First columns to drop
del_cols = ['REPORT_NO', 'INJ_SEVER_CODE', 'PERSON_ID', 'PERSON_TYPE', 'VEHICLE_ID', 'TIME']
# Drop columns
ddf = cdf.drop(del_cols, axis=1)

#### Reasons for neglection
REPORT_NO: Report number is not of concern.  
INJ_SEVER_CODE: This attribute will be used for analysis.  
PERSON_ID: Individual ID is not of concern.  
PERSON_TYPE: Every record already satisfies PERSON_TYPE == 'D'.  
VEHICLE_ID: Vehicle ID is not of concern.  
TIME: Since there is no attribute that can tell the traffic situation for each hour, day or month span, TIME attribute can only indicate environmental conditions which are indicated in other attributes. Hence, this attribute is unncessary.

In [8]:
# List of columns to delete
del_skew = []

# Get column names as list
col_list = ddf.columns

# Identify columns with skewed data
for col in col_list:
    d = dict(collections.Counter(cdf['{}'.format(col)]))
    # Key with max value
    max_key = max(d, key=d.get)
    # Max value
    max_val = d[max_key]
    # Percentage of max value
    occ = max_val / len(ddf)
    
    print('{c}:\n\t[Max] {k}: {p}'.format(c=col, k=max_key, p=occ))
    
    # If the largest data of a column occupy more than 70%, assume the column skewed
    if occ > 0.7:
        # Add the column name in a list for to-be-deleted
        del_skew.append(col)
        print('!!Data is greatly skewed.!!')
    
    print('')

CDL_FLAG:
	[Max] N: 0.9596864123272395
!!Data is greatly skewed.!!

CONDITION_CODE:
	[Max] Apparently Normal: 0.9597539958774034
!!Data is greatly skewed.!!

EQUIP_PROB_CODE:
	[Max] No Misuse: 0.9966208224918055
!!Data is greatly skewed.!!

FAULT_FLAG:
	[Max] N: 0.58547629506978

SAF_EQUIP_CODE:
	[Max] Shoulder/Lap Belt(s): 0.9799952691514885
!!Data is greatly skewed.!!

SEX_CODE:
	[Max] F: 0.5468185043760349

COLLISION_TYPE_CODE:
	[Max] Same Direction Rear End: 0.34937316257222994

C_M_ZONE_FLAG:
	[Max] N: 0.9844895752373872
!!Data is greatly skewed.!!

JUNCTION_CODE:
	[Max] Non Intersection: 0.5374243909032541

LANE_CODE:
	[Max] Right Turn Lane: 0.6486669144730173

LIGHT_CODE:
	[Max] Daylight: 0.6134896766127125

RD_COND_CODE:
	[Max] No Defects: 0.9863143310918122
!!Data is greatly skewed.!!

RD_DIV_CODE:
	[Max] Two-way, Not Divided: 0.4170242962862839

SURF_COND_CODE:
	[Max] Dry: 0.7480147332139357
!!Data is greatly skewed.!!

WEATHER_CODE:
	[Max] Clear: 0.7142567499070727
!!Data is

In [9]:
# Drop columns of skewed data
ddf2 = ddf.drop(del_skew, axis=1)

#### Reasons for neglection
Below attributes are dropped due to being highly skewed.  
CDL_FLAG: 'N' 95%  
CONDITION_CODE: 'Apparently Normal' 95%  
EQUIP_PROB_CODE: 'No Misuse' 99%  
SAF_EQUIP_CODE:'Shoulder/Lap Belts(s)' 97%  
C_M_ZONE_FLAG: 'N' 98%  
RD_COND_CODE: 'No Defects' 98%  
SURF_COND_CODE: 'Dry' 74%  
WEATHER_CODE: 'Clear' 71%  
HIT_AND_RUN_FLAG: 'N' 98%

#### Reasons for neglection
REPORT_NO: Report number is not of concern.  
CDL_FLAG: Data is skewed.  
EQUIP_PROB_CODE: Safety equipment problem does not cause an accident.  
FAULT_FLAG: Every record already satisfies FAULT_FLAG == 'Y'.  
PERSON_ID: Individual ID is not of concern.  
PERSON_TYPE: Every record already satisfies PERSON_TYPE == 'D'.  
VEHICLE_ID: Vehicle ID is not of concern.  
COLLISION_TYPE_CODE: How the cars collided is not of concern.  
C_M_ZONE_FLAG: Data is skewed.  
JUNCTION_CODE: Road structure is not of concern.  
LANE_CODE: Road structure is not of concern.  
RD_DIV_CODE: Road structure is not of concern.  
SURF_COND_CODE: Surface condition may imply driver's state. However, it is hard to depict from this attribute.  
AREA_DAMAGED_CODE_MAIN: Which part of the car is damaged is not of concern.  
DAMAGE_CODE: Since casualty is expressed on INJ_SEVER_CODE, the amount of damage on the car is unnecessary.  
HIT_AND_RUN_FLAG: Data is skewed. Also, hit-and-run happens after the collision. It seems unlikely it will represent the driver's state before collision.  
TIME: Since there is no attribute that can tell the traffic situation for each hour, day or month span, TIME attribute can only indicate environmental conditions which are indicated in other attributes. Hence, this attribute is unncessary.

In [ ]:
# Drop columns
drop_df = fault_df.drop(del_cols, axis=1)

In [ ]:
# Get all left column names
drop_df.columns

In [ ]:
# Change categorical values with only two options to numerical values
d = {
    'SEX_CODE': {'F': 0, 'M': 1},
}

rdf = drop_df.replace(d)

In [ ]:
# Scale AGE
# Since AGE has values greater than any other column, its values should be scaled
scaled_df = rdf.copy()
scaled_df['AGE'] = (rdf['AGE'] - rdf['AGE'].min()) / (rdf['AGE'].max() - rdf['AGE'].min())
# scaled_df['AGE'] = (rdf['AGE'] - rdf['AGE'].mean()) / rdf['AGE'].std()

In [ ]:
# Dummify all categorical data except INJ_SEVER_CODE
dum_cols = [
    'CONDITION_CODE', 'SAF_EQUIP_CODE','LIGHT_CODE',
    'RD_COND_CODE', 'WEATHER_CODE', 'BODY_TYPE_CODE', 'MOVEMENT_CODE'
]

dum_df = pd.get_dummies(scaled_df, columns=dum_cols)

In [ ]:
# Separate INJ_SEVER_CODE
preprocessed_df = dum_df.drop('INJ_SEVER_CODE', axis=1)

In [ ]:
# To make all columns shown, find out the number of columns
n = len(dum_df.columns)
# Change display setting
pd.options.display.max_columns = n

### Clustering

#### k-means

In [ ]:
# Import package
from sklearn.cluster import KMeans

In [ ]:
# Fit
n_clusters = 5
kmodel = KMeans(n_clusters=n_clusters)
kmodel.fit(preprocessed_df)

In [ ]:
# Predicted labels
predicted_labels = kmodel.labels_
drop_df['kmeans_labels'] = predicted_labels

In [ ]:
# See if labels are skewed
drop_df.groupby('kmeans_labels').count()

#### Silhouette

In [ ]:
# Import packages
from sklearn.metrics import silhouette_score, silhouette_samples
from matplotlib import cm
from matplotlib import pyplot as plt

import numpy as np

In [ ]:
# Silhouette average
savg = silhouette_score(preprocessed_df, drop_df['kmeans_labels'])

In [ ]:
# Silhouette value of each data
sval = silhouette_samples(preprocessed_df, drop_df['kmeans_labels'])

In [ ]:
def plot_silhouette(X_df, labels):
    cluster_labels = sorted(np.unique(labels))
    n_clusters = len(cluster_labels)
    
    plt.figure()

    # axis setting
    ax = plt.gca()
    ax.set_xlim([-0.5, 1])
    ax.set_ylim([0, X_df.shape[0] + (n_clusters + 1) * 10])

    y_lower = 10
    
    silhouette_avg = silhouette_score(X_df, labels)
    print("k : {}".format(n_clusters))
    print("silhouette_avg : {}".format(silhouette_avg))
    data_silhouette_values = silhouette_samples(X_df, labels)

    for i in cluster_labels:
        # 특정 클러스터의 silhouette 값만 추출
        ith_cluster_silhouette_values = data_silhouette_values[labels == i]
        
        # 내림차순으로 정렬
        ith_cluster_silhouette_values.sort()

        # 해당 클러스터의 크기
        size_cluster_i = len(ith_cluster_silhouette_values)
       
        # 클러스터의 silhouette을 표시할 y축 최고값 결정
        y_upper = y_lower + size_cluster_i

        # 색
        color = cm.spectral(float(i) / n_clusters)
        
        # plot silhouette
        ax.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # y축에 클러스터 이름 표시
        ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # 다음 클러스터의 silhouette을 표시할 최저점 조정
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax.set_title("The silhouette plot for k={}".format(n_clusters))
    ax.set_xlabel("The silhouette coefficient values")
    ax.set_ylabel("Cluster label")

    # silhouette 평균 값을 나타내는 선
    ax.axvline(x=silhouette_avg, color="red", linestyle="--")

    # y축 값 제거
    ax.set_yticks([])

    plt.show() 

In [ ]:
# Plot silhouette for choosing k
k_candidates = [2, 3, 4, 5, 6, 7]

for i in k_candidates:
    ith_model = KMeans(n_clusters=i)
    labels = ith_model.fit_predict(preprocessed_df)
    plot_silhouette(preprocessed_df, labels)

### Classifying

#### Decision tree

In [ ]:
# Import packages
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Split X, y
X, y = preprocessed_df, drop_df.kmeans_labels

In [ ]:
# Split train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [ ]:
# Model
class_weight = 'balanced'
max_leaf_nodes = 5
dt_model = DecisionTreeClassifier(class_weight=class_weight, max_leaf_nodes=max_leaf_nodes)
# max_depth = 5
# dt_model = DecisionTreeClassifier(class_weight=class_weight, max_depth=max_depth)
dt_model.fit(X=X_train, y=y_train)

In [ ]:
# Get predictions
y_pred_dt = dt_model.predict(X=X_test)

In [ ]:
# Get score
dt_model.score(X=X_test, y=y_test)

#### Graphviz

In [ ]:
# Import package
from sklearn.tree import export_graphviz

import graphviz

In [ ]:
# Export tree as .dot
with open('test_tree.dot', 'w') as f:
    export_graphviz(dt_model, f, feature_names=preprocessed_df.columns)

In [ ]:
# Open .dot and visualize
with open('test_tree.dot', 'r') as f:
    dot_graph = f.read()

graphviz.Source(dot_graph)

#### Subclass k-means

In [ ]:
# Copy dataframe for subclass
sub_df = scaled_df.copy()
sub_df = sub_df[sub_df['MOVEMENT_CODE'] == 'Moving Constant Speed']
sub_df = sub_df.drop('MOVEMENT_CODE', axis=1)

In [ ]:
# Dummify all categorical data except INJ_SEVER_CODE
sub_dum_cols = [
    'CONDITION_CODE', 'EQUIP_PROB_CODE', 'SAF_EQUIP_CODE','LIGHT_CODE',
    'RD_COND_CODE', 'WEATHER_CODE', 'BODY_TYPE_CODE',
]

sub_dum_df = pd.get_dummies(sub_df, columns=sub_dum_cols)

In [ ]:
# Separate INJ_SEVER_CODE
sub_preprocessed_df = sub_dum_df.drop('INJ_SEVER_CODE', axis=1)

In [ ]:
# Fit
n_clusters = 5
kmodel = KMeans(n_clusters=n_clusters)
kmodel.fit(sub_preprocessed_df)

In [ ]:
# Predicted labels
predicted_labels = kmodel.labels_
sub_df['sub_kmeans_labels'] = predicted_labels

#### Subclass decision tree

In [ ]:
# Split X, y
X, y = sub_preprocessed_df, sub_df.sub_kmeans_labels

In [ ]:
# Split train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

In [ ]:
# Model
class_weight = 'balanced'
max_leaf_nodes = 5
dt_model = DecisionTreeClassifier(class_weight=class_weight, max_leaf_nodes=max_leaf_nodes)
# max_depth = 5
# dt_model = DecisionTreeClassifier(class_weight=class_weight, max_depth=max_depth)
dt_model.fit(X=X_train, y=y_train)

In [ ]:
# Get predictions
y_pred_dt = dt_model.predict(X=X_test)

In [ ]:
# Get score
dt_model.score(X=X_test, y=y_test)

#### Subclass graphviz

In [ ]:
# Export tree as .dot
with open('test_tree_sub.dot', 'w') as f:
    export_graphviz(dt_model, f, feature_names=sub_preprocessed_df.columns)

In [ ]:
# Open .dot and visualize
with open('test_tree_sub.dot', 'r') as f:
    dot_graph = f.read()

graphviz.Source(dot_graph)